In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [3]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [6]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
count_vectorizer = feature_extraction.text.CountVectorizer()

example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [8]:
print(example_train_vectors[0].todense().shape)

(1, 54)


In [9]:
print(example_train_vectors[0].todense())

[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [10]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])
test_vectors = count_vectorizer.transform(test_df["text"])

In [11]:
clf = linear_model.RidgeClassifier()

In [12]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring='f1')
scores

array([0.59421842, 0.56498283, 0.64113893])

### RNN

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
vocab_size = 10000
embedding_size = 100
max_length = 100
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=embedding_size,
                            input_length=max_length)

In [15]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_df["text"])
train_sequences = tokenizer.texts_to_sequences(train_df["text"])
test_sequences = tokenizer.texts_to_sequences(test_df["text"])

In [16]:
train_sequences_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

In [17]:
model = tf.keras.Sequential([
    embedding_layer,
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_sequences_padded, train_df['target'], epochs=5, batch_size=32)

Epoch 1/5
238/238 [==============================] - 7s 23ms/step - loss: 0.6119 - accuracy: 0.6782
Epoch 2/5
238/238 [==============================] - 5s 23ms/step - loss: 0.4539 - accuracy: 0.8144
Epoch 3/5
238/238 [==============================] - 5s 23ms/step - loss: 0.3667 - accuracy: 0.8542
Epoch 4/5
238/238 [==============================] - 5s 22ms/step - loss: 0.3006 - accuracy: 0.8844
Epoch 5/5
238/238 [==============================] - 5s 22ms/step - loss: 0.2357 - accuracy: 0.9105


In [19]:
pred = model.predict(test_sequences_padded)

102/102 [==============================] - 1s 6ms/step


In [20]:
pred.shape

(3263, 1)

In [21]:
pred = pred[:,0]

In [22]:
pred

array([0.463321  , 0.38027593, 0.6462529 , ..., 0.7705619 , 0.8710564 ,
       0.8972658 ], dtype=float32)

In [23]:
threshold = 0.5
binary_pred = (pred >= threshold).astype(int)

In [24]:
output_df = pd.DataFrame({'id': test_df['id'], 'target': binary_pred})
output_df.to_csv('submission.csv', index=False)